## 安装依赖

In [ ]:
!pip install -qU canopy-sdk

In [ ]:
!pip install -qU langchain langchain_openai cohere==4.27 markdown google-search-results

  Preparing metadata (setup.py) ... done


- 如果你希望在Colab上运行Canopy则运行下述代码

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 29.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-stubs 2.2.1.240316 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.24.4 which is incompatible.


## Credentials

In [ ]:
# 加载各种Key
import getpass
import os
print("Provide your Pinecone key")
os.environ["PINECONE_API_KEY"] =  getpass.getpass()
print("Provide your OpenAI API key")
os.environ["OPENAI_API_KEY"] = getpass.getpass()
print("Provide your Serp API key")
os.environ["SERPAPI_API_KEY"] = getpass.getpass()
print("Provide your Cohere API key")
os.environ["COHERE_API_KEY"] = getpass.getpass()

## Read data

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# 读取PineCone的文档数据
data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


## 初始化 Canopy

In [ ]:
from canopy.knowledge_base import KnowledgeBase
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

INDEX_NAME = "advanced-rag"
kb = KnowledgeBase(index_name=INDEX_NAME)


- 如果没有创建过Canopy index，则创建一个 Canopy index

In [ ]:
from canopy.knowledge_base import list_canopy_indexes
if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index()

- 将上述导入的数据中的行数据转化为 Canopy 的文档数据

In [ ]:
from canopy.models.data_models import Document
documents = [Document(**row) for _, row in data.iterrows()]

- 对文档进行Upsert操作，塞入Canopy的知识库

In [ ]:
from tqdm.auto import tqdm
kb = KnowledgeBase(index_name=INDEX_NAME)
kb.connect()
batch_size = 10

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/6 [00:00<?, ?it/s]

- 测试Canopy的知识库

In [ ]:
from canopy.models.data_models import Query

results = kb.query([Query(text="p1 pod capacity")])

In [ ]:
results

## 初始化上下文和聊天引擎

In [ ]:
# 初始化上下文引擎
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [ ]:
# 初始化聊天引擎
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

In [ ]:
# 定义聊天对话函数
from typing import Tuple
from canopy.models.data_models import Messages, UserMessage, AssistantMessage

def chat(new_message: str, history: Messages) -> Tuple[str, Messages]:
    messages = history + [UserMessage(content=new_message)] # 历史数据 + 用户信息向大模型的作为输入
    response = chat_engine.chat(messages)
    assistant_response = response.choices[0].message.content
    return assistant_response, messages + [AssistantMessage(content=assistant_response)]

In [ ]:
# 将string类型的数据转化为Json数据
import json
import ast

def str_to_json(s):
    try:
        # First, attempt to parse the string as JSON
        return json.loads(s)
    except json.JSONDecodeError:
        # If it fails, assume the string might be a Python literal
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            # Handle the case where parsing fails for both methods
            print("Error: Input string is neither valid JSON nor a valid Python literal.")
            return None



## 评估大模型

- 在这里，我们调用大型语言模型（LLM）本身，以不同的方式评估相对于特定提示生成的文本

In [ ]:
def evaluate_with_llm(model, prompt, generated_text):
    """
    Uses a Large Language Model (LLM) to evaluate generated text.

    :param model: An instance of the LLM, ready to generate responses.
    :param prompt: The original prompt given to the system.
    :param generated_text: The text generated by the SELF-RAG system.
    :return: A dictionary containing critique scores or assessments.
    """
    evaluations = {}

    # 从提供的模板和关键字参数中创建评估查询
    def create_evaluation_query(template, **kwargs):
        query = ChatPromptTemplate.from_template(template)
        chain = query | model
        return float(chain.invoke(kwargs).content)

    # 相关性评估
    relevance_template = "Given the context provided by the following prompt: '{prompt}', please evaluate on a scale from 0 to 1, where 1 is highly relevant and 0 is not relevant at all, how relevant is this generated response: '{generated_text}'? Provide a numerical score only."
    evaluations['relevance'] = create_evaluation_query(relevance_template, prompt=prompt, generated_text=generated_text)

    # 清晰度评估
    clarity_template = "How clear and easily understandable is this text: '{generated_text}'? Rate its clarity on a scale from 0 to 1, where 1 indicates that the text is very clear and 0 indicates that the text is very unclear. Provide a numerical score only."
    evaluations['clarity'] = create_evaluation_query(clarity_template, prompt=prompt, generated_text=generated_text)

    # 连贯性评估
    coherence_template = "On a scale from 0 to 1, with 1 being highly coherent and 0 being not coherent at all, how well do the ideas in this generated text: '{generated_text}' flow together? Consider if the text makes logical sense as a whole. Provide a numerical score only."
    evaluations['coherence'] = create_evaluation_query(coherence_template, prompt=prompt, generated_text=generated_text)

    # 详尽性评估
    detail_template = "Assessing the detail and exhaustiveness relative to the prompt '{prompt}', how thoroughly does this generated text: '{generated_text}' cover the topic? Rate on a scale from 0 to 1, where 1 is very detailed and exhaustive, and 0 is not detailed at all. Provide a numerical score only."
    evaluations['details'] = create_evaluation_query(detail_template, prompt=prompt, generated_text=generated_text)

    # 作为答案的适宜性评估
    suitability_template = "Evaluate the suitability of this generated text: '{generated_text}' as an answer to the original prompt '{prompt}'. On a scale from 0 to 1, where 1 is a perfect answer and 0 is completely unsuitable, provide a numerical score only."
    evaluations['suitability'] = create_evaluation_query(suitability_template, prompt=prompt, generated_text=generated_text)

    return evaluations


## Critique
- 评价函数创建评估并根据可配置的权重产生加权平均值
- 通过这种方式，可以改变每个评估得分对最终评价分数的影响

In [ ]:
def critique(model, prompt, generated_text):
    evaluation_weights = {
        'relevance': 3,
        'clarity': 1,
        'coherence': 0.5,
        'details': 1.5,
        'suitability': 2
    }

    # 调用函数 evaluate_with_llm 来获取各维度的评估分数
    evaluations = evaluate_with_llm(model, prompt, generated_text)
    print("Evaluations:", evaluations)

    # 计算评估得分的加权和
    weighted_sum = sum(evaluations[aspect] * evaluation_weights.get(aspect, 1) for aspect in evaluations)

    # 计算评估的权重总和
    total_weight = sum(evaluation_weights.get(aspect, 1) for aspect in evaluations)

    # 计算评估得分的加权平均值
    weighted_average = weighted_sum / total_weight if total_weight > 0 else 0

    # 返回加权平均值和各维度的评估得分
    return [weighted_average, evaluations]


In [ ]:
def is_retrieval_needed(model, prompt):
  # 定义一个基于模板的提示，询问是否需要从外部来源检索信息来回答问题
  is_retrieval_needed_prompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', is retrieval from an external source necessary to answer the question? Reply with only True or False")
  # 创建一个查询链，该链首先应用模板，然后传递给模型进行处理
  is_retrieval_needed_chain = is_retrieval_needed_prompt | model
  # 执行查询链，并传入实际的提示文本，获取模型的回答
  return is_retrieval_needed_chain.invoke({"prompt": prompt}).content

In [ ]:
def consolidate(model, text):
  consolidate_prompt = ChatPromptTemplate.from_template("Given the following set of texts, please consolidate them: '{text}'")
  consolidate_chain = consolidate_prompt | model

  return consolidate_chain.invoke({"text": text}).content

In [ ]:
def compare(model, query, text1, text2):
  # 创建一个基于模板的聊天提示，指示模型整合提供的文本集
  compare_prompt = ChatPromptTemplate.from_template("Given the following query: '{query}', score text1 and text2 between 0 and 1, to indicate which provides a better answer overall to the query. Reply with two numbers in an array, for example: [0.1, 0.9]. The sum total of the values should be 1. text1: '{text1}' \n text2: '{text2}'")
  # 创建一个查询链，该链首先应用整合模板，然后将其传递给模型处理
  compare_chain = compare_prompt | model
  # 执行查询链，并传入实际的文本，获取模型整合后的内容
  return str_to_json(compare_chain.invoke({"query": query, "text1": text1, "text2": text2}).content)

In [ ]:
def generate_queries(model,prompt, num_queries):
  # 创建一个基于模板的聊天提示，指示模型根据给定的提示生成指定数量的问题
  query_generation_prompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
  # 创建一个查询链，该链首先应用生成问题的模板，然后将其传递给模型处理
  query_generation_chain = query_generation_prompt | model
  # 执行查询链，并传入实际的提示和问题数量，获取模型生成的问题列表
  return str_to_json(query_generation_chain.invoke({"prompt": prompt, "num_queries": num_queries}).content)

A helper function to get text from an array of Canopy `Document` objects.

In [ ]:
from canopy.models.data_models import Query

def extract_documents_texts(results):
    # 初始化一个空列表，用于存储提取的文本
    all_texts = []
    # 遍历结果列表中的每个QueryResult
    for result in results:
        # 假设result.documents是访问QueryResult中文档的正确方式
        for document in result.documents:
            # 假设document.text是访问DocumentWithScore中文本的正确方式
            all_texts.append(document.text)
    # 返回所有文本的平面列表
    return all_texts



A reranking function that queries the knowledge base, then reranks the results using Cohere's `rerank-english-v2.0` model.

In [ ]:
import cohere
from langchain_community.utilities import SerpAPIWrapper
# 创建Cohere客户端，使用环境变量中的API密钥
co = cohere.Client(os.environ["COHERE_API_KEY"])


def get_reranked_result(query, top_n=1):
  # 使用知识库查询输入的查询词，并返回匹配结果
  matches = kb.query([Query(text=query)])
  # 使用先前定义的函数从匹配结果中提取所有文档的文本
  docs = extract_documents_texts(matches)
  # 使用Cohere的重排序模型对文档进行重排序，获取与查询最相关的顶部文档
  rerank_results = co.rerank(model="rerank-english-v2.0", query=query, documents=docs, top_n=top_n)
  texts = []
  # 遍历重排序的结果
  for rerank_result in rerank_results:
      # 从每个重排序结果中访问'document'属性中的'text'字段
      text = rerank_result.document['text']
      texts.append(text)
  return texts

In [ ]:
from typing import List, Dict, Any, Tuple
from collections import defaultdict

class QueryDetail:
    def __init__(self, query: str):
        self.query = query
        self.content: List[str] = []
        self.critique_score: float = 0.0
        self.critique_details: Dict[str, Any] = {}
        self.retrieval_needed: bool = False
        self.search_needed: bool = False

    def add_response(self, model, search) -> None:
        """处理查询以添加回应，处理检索和批评。"""
        if is_retrieval_needed(model, self.query):
            response = " ".join(get_reranked_result(self.query, top_n=3))
            self.retrieval_needed = True
        else:
            response = "Some generated answer"
            self.retrieval_needed = False

        self.content.append(response)

        critique_score, critique_details = critique(model, self.query, response)
        self.critique_score = critique_score
        self.critique_details = critique_details
        self.search_needed = critique_score < 0.5

        if self.search_needed:
            self.search_and_add_results(search)

    def search_and_add_results(self, search) -> None:
        """如果批评得分低，执行搜索并处理结果。"""
        search_result_raw = search.run(self.query)
        search_result = str_to_json(search_result_raw) or []
        self.content.extend(search_result)

class QueryProcessor:
    def __init__(self, model, search, queries: List[str]):
        self.model = model
        self.search = search
        self.queries = [QueryDetail(query) for query in queries]

    def process_queries(self) -> List[QueryDetail]:
        """处理列表中的每个查询。"""
        for query_detail in self.queries:
            query_detail.add_response(self.model, self.search)
            if query_detail.search_needed:
                consolidated_response = consolidate(self.model, query_detail.content)
                query_detail.content = [consolidated_response]
                critique_score, critique_details = critique(self.model, query_detail.query, consolidated_response)
                query_detail.critique_score = critique_score
                query_detail.critique_details = critique_details
        return self.queries

def advanced_rag_query(model, query: str, num_queries: int) -> List[QueryDetail]:
    search = SerpAPIWrapper()
    initial_queries = generate_queries(model, query, num_queries)[:num_queries]
    query_processor = QueryProcessor(model, search, initial_queries)
    processed_queries = query_processor.process_queries()
    return processed_queries


In [ ]:
query = "How can I make a new Pinecone index?"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(model="gpt-4")

In [ ]:
results = advanced_rag_query(model, query, 3)
combined_content = " ".join(content for result in results for content in result.content)
advanced_rag_results = consolidate(model, combined_content)

Evaluations: {'relevance': 0.875, 'clarity': 0.8, 'coherence': 0.9, 'details': 0.8, 'suitability': 0.85}
Evaluations: {'relevance': 1.0, 'clarity': 0.8, 'coherence': 0.8, 'details': 0.9, 'suitability': 0.8}
Evaluations: {'relevance': 1.0, 'clarity': 0.8, 'coherence': 0.9, 'details': 0.9, 'suitability': 0.9}


In [ ]:
advanced_rag_critique = critique(model, query, advanced_rag_results)

Evaluations: {'relevance': 1.0, 'clarity': 0.85, 'coherence': 1.0, 'details': 0.8, 'suitability': 1.0}


In [ ]:
history = []
rag_result, history = chat(query, history)
rag_critique = critique(model, query, rag_result)


Evaluations: {'relevance': 1.0, 'clarity': 0.8, 'coherence': 1.0, 'details': 0.9, 'suitability': 1.0}


In [ ]:
from IPython.display import display, HTML
import markdown

# Convert your Markdown content and critiques to strings, assuming they might not be strings already
final_result_html = markdown.markdown(str(advanced_rag_results))
rag_result_html = markdown.markdown(str(rag_result))
advanced_rag_critique_html = markdown.markdown(str(advanced_rag_critique))
rag_critique_html = markdown.markdown(str(rag_critique))

# Construct HTML table with the pre-rendered HTML content for each cell
html = f"""
<table>
<tr>
    <th>Advanced RAG</th>
    <th>RAG</th>
</tr>
<tr>
    <td>{final_result_html}</td>
    <td>{rag_result_html}</td>
</tr>
<tr>
    <td>{advanced_rag_critique_html}</td>
    <td>{rag_critique_html}</td>
</tr>
</table>
"""

# Display the HTML table
display(HTML(html))


Advanced RAG,RAG
"To create an index in Pinecone, first download a pre-embedded dataset from the pinecone-datasets library. This allows you to skip the embedding and preprocessing steps. ```python import pinecone_datasets dataset = pinecone_datasets.load_dataset('wikipedia-simple-text-embedding-ada-002-100K') dataset.head() ``` After downloading the data, initialize your Pinecone environment and create your first index. You have the option to select a distance metric for your index. Note - By default, all fields are indexed. To avoid redundant and costly indexing, pass an additional empty metadata_config parameter. python pinecone.create_index( name=index_name_v1, metric='cosine', dimension=1536, metadata_config={“indexed”:[]} ) Before creating an index, ensure your Pinecone API key is set up. If the 'openai' index already exists, you can connect to it directly. ```python import pinecone pinecone.init( api_key=""YOUR_API_KEY"", environment=""YOUR_ENV"" ) if 'openai' not in pinecone.list_indexes(): pinecone.create_index('openai', dimension=len(embeds[0])) index = pinecone.Index('openai') ``` When preparing your project structure, consider creating separate projects for your development and production indexes. This allows you to test changes before deploying them to production. Ensure that you have properly configured user access to your production environment. Before moving your index to production, test that your index is returning accurate results in the context of your application. Consider identifying the appropriate metrics for evaluating your results.","To create a new Pinecone index, you can follow these general steps: Initialize a connection to Pinecone using your API key and environment. Check if the index already exists, and if not, create a new index with a specified dimension. Connect to the newly created index for further operations. You can also consider specifying additional configurations and choices like the distance metric and metadata fields during the index creation process."
"[0.94375, {'relevance': 1.0, 'clarity': 0.85, 'coherence': 1.0, 'details': 0.8, 'suitability': 1.0}]","[0.95625, {'relevance': 1.0, 'clarity': 0.8, 'coherence': 1.0, 'details': 0.9, 'suitability': 1.0}]"


In [ ]:
compare(model, query, advanced_rag_results, rag_result)

[0.9, 0.1]